***Гибридный чат-бот***

***Если запрос будет состоять меньше чем из трех слов, то использоваться будет TF-IDF (Lionelya Pepsi), иначе будет использоваться чат-бот, основанный на трансформере***

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pickle
import csv
import random
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
def talk_to_lionelya_pepsi_primary(test_set_sentence):
    csv_file_path = "DataSet/chat_bot_df.csv"
    tfidf_vectorizer_pikle_path = "DataSet/tfidf_vectorizer.pickle"
    tfidf_matrix_train_pikle_path ="DataSet/tfidf_matrix_train.pickle"

    try:
        with open(tfidf_vectorizer_pikle_path, 'rb') as f:
            tfidf_vectorizer = pickle.load(f)
        with open(tfidf_matrix_train_pikle_path, 'rb') as f:
            tfidf_matrix_train = pickle.load(f)
    except FileNotFoundError:
        sentences = []
        with open(csv_file_path, "r", encoding='utf-8') as sentences_file:
            reader = csv.reader(sentences_file, delimiter=',')
            for row in reader:
                sentences.append(row[0])

        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix_train = tfidf_vectorizer.fit_transform(sentences)

        with open(tfidf_vectorizer_pikle_path, 'wb') as f:
            pickle.dump(tfidf_vectorizer, f)
        with open(tfidf_matrix_train_pikle_path, 'wb') as f:
            pickle.dump(tfidf_matrix_train, f)

    test_set = (test_set_sentence, "")
    tfidf_matrix_test = tfidf_vectorizer.transform(test_set)
    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)
    cosine = np.delete(cosine, 0)

    max_similarity = cosine.max()
    response_index = 0
    if max_similarity > 0.7:
        new_max = max_similarity - 0.01
        idx = np.where(cosine > new_max)
        response_index = random.choice(idx[0])
    else:
        response_index = np.where(cosine == max_similarity)[0][0] + 2

    with open(csv_file_path, "r", encoding='utf-8') as sentences_file:
        reader = csv.reader(sentences_file, delimiter=',')
        for idx, row in enumerate(reader, 1):
            if idx == response_index:
                return row[1], response_index
            
model_name = "gpt2"
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def generate_transformer_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    response = gpt2_model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    response_text = tokenizer.decode(response[0], skip_special_tokens=True)
    return response_text

def generate_hybrid_response(input_text):
    if len(input_text.split()) < 3:
        response = talk_to_lionelya_pepsi_primary(input_text)
    else:
        response = generate_transformer_response(input_text)
    return response

while True:
    user_input = input("Ваш вопрос: ")
    response = generate_hybrid_response(user_input)
    print("Ответ чат-бота:", response)